# 9章 潜在顧客を把握するための画像認識10本ノック

In [1]:
# 下準備として，データのあるディレクトリに移動しておく
import os

DATA_ROOT_DIR = "./sample/9章/"
os.chdir(DATA_ROOT_DIR)

## ノック81 画像データを読み込んでみよう
まずはデータを読み込む．

In [2]:
import cv2
from IPython.display import Image, display
img = cv2.imread("img/img01.jpg")
height, width = img.shape[:2]
print("画像幅: " + str(width))
print("画像高さ: " + str(height))

# display(Image("img/img01.jpg"))

画像幅: 1920
画像高さ: 1440


## ノック８２：映像データを読み込んでみよう
やはりデータを読み込む．

In [3]:
import cv2
from IPython.display import Video, display, YouTubeVideo, HTML

# 情報取得
cap = cv2.VideoCapture("mov/mov01.avi")
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
count = cap.get(cv2.CAP_PROP_FRAME_COUNT)
fps = cap.get(cv2.CAP_PROP_FPS)
print("画像幅: " + str(width))
print("画像高さ: " + str(height))
print("総フレーム数: " + str(count))
print("FPS: " + str(fps))

# 出力
# while(cap.isOpened()):
#     ret, frame = cap.read()
#     if ret:
#         cv2.imshow("frame", frame)
#     if cv2.waitKey(1) & 0xFF == ord("q"):
#         break
cap.release()
# cv2.destroyAllWindows()

# display(Video("mov/mov01.avi", embed=True))

画像幅: 1920.0
画像高さ: 1440.0
総フレーム数: 401.0
FPS: 30.0


## ノック83 映像を画像に分割し、保存してみよう
動画から画像を切り抜く



In [4]:
import cv2
import os
cap = cv2.VideoCapture("mov/mov01.avi")
num = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        # cv2.imshow("frame", frame)
        filepath = "snapshot/snapshot_" + str(num) + ".jpg"
        if not os.path.exists(filepath):
            cv2.imwrite(filepath,frame)
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break
    else:
        break
    num = num + 1
cap.release()
# cv2.destroyAllWindows()

## ノック84 画像内のどこに人がいるのかを検出してみよう
HOG特徴量を利用して人の検出を行う．

In [5]:
import cv2
from IPython.display import Image, display

# 準備
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
hogParams = {'winStride': (8, 8), 'padding': (32, 32), 'scale': 1.05, 'hitThreshold':0, 'finalThreshold':5}

# 検出
img = cv2.imread("img/img01.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
human, r = hog.detectMultiScale(gray, **hogParams)
if (len(human)>0):
    for (x, y, w, h) in human:
        cv2.rectangle(img, (x, y), (x + w, y + h), (255,255,255), 3)
# cv2.imshow("img",img)
_ = cv2.imwrite("temp.jpg",img)
# display(Image("temp.jpg"))

## ノック85 画像内の人の顔を検出してみよう
Haar-like特徴量を利用して顔検出を行う．

In [6]:
import cv2
from IPython.display import Image, display

# 準備
cascade_file = "haarcascade_frontalface_alt.xml"
cascade = cv2.CascadeClassifier(cascade_file)

# 検出
img = cv2.imread("img/img02.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
face_list = cascade.detectMultiScale(gray, minSize=(50, 50))

for (x, y, w, h) in face_list:
    color = (0, 0, 225)
    pen_w = 3
    cv2.rectangle(img, (x, y), (x+w, y+h), color, thickness = pen_w)

# cv2.imshow("img",img)
_ = cv2.imwrite("temp.jpg",img)
# cv2.waitKey(0)
# display(Image("temp.jpg"))